# Data Preparation (from .xml to .tfrecord)

In [1]:
import os, glob, pandas as pd, xml.etree.ElementTree as ET, tensorflow as tf, cv2, numpy as np

from collections import namedtuple

In [2]:
def read_label_map(label_map_path):
    item_id = None
    item_name = None
    items = {}

    with open(label_map_path, "r") as file:
        for line in file:
            line.replace(" ", "")
            if line == "item{":
                pass
            elif line == "}":
                pass
            elif "id" in line:
                item_id = int(line.split(":", 1)[1].strip())
            elif "name" in line:
                item_name = line.split(":", 1)[1].replace("'", "").replace("\"", "").strip()

            if item_id is not None and item_name is not None:
                items[item_name] = item_id
                item_id = None
                item_name = None

    return items

In [3]:
label_map = "./label_map.pbtxt"
label_map_dict = read_label_map(label_map)

label_map_dict

{'pothole': 1, 'crack': 2}

In [4]:
def class_text_to_num(text):
    return label_map_dict[text]

In [5]:
def xml_to_csv(path):
    xml_list = []
    for file in glob.glob(path + "/*.xml"):
        tree = ET.parse(file)
        root = tree.getroot()
        for member in root.findall("object"):
            if member[0].text == "plain": continue
            print(root.find("filename").text)
            xml_list.append((
                root.find("filename").text,
                int(member[4][0].text),
                int(member[4][1].text),
                int(member[4][2].text),
                int(member[4][3].text),
                int(class_text_to_num(member[0].text)),
            ))  # filename, x1, y1, x2, y2, class
    columns = ["filename", "x1", "y1", "x2", "y2", "class"]
    return pd.DataFrame(xml_list, columns = columns)

In [6]:
def regroup(dataframe, group = "filename"):
    data = namedtuple("data", ["filename", "objects"])
    group_by = dataframe.groupby(group)
    return [data(filename, group_by.get_group(x)) for filename, x in zip(group_by.groups.keys(), group_by.groups)]

In [7]:
def bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    # If the value is an eager tensor BytesList won't unpack a string from an EagerTensor.
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [8]:
def float_feature(value):
    """Returns a bytes_list from a string / byte."""
    # If the value is an eager tensor BytesList won't unpack a string from an EagerTensor.
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() 
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

In [9]:
def create_tf_ex(group, path):
    image = cv2.imread(os.path.join(path, group.filename), cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    filename = group.filename.encode("utf-8")
    label = []
    bbox = []
    for i, data in group.objects.iterrows():
        bbox.append([float(data["y1"] / 512), float(data["x1"] / 512), float(data["y2"] / 512), float(data["x2"] / 512)])
        label.append(data["class"])
    bbox = np.array(bbox)
    bbox = bbox.astype("float32")
    features = tf.train.Features(feature = {
        "filename": bytes_feature(filename),
        "pic": bytes_feature(tf.io.serialize_tensor(image)),
        "bbox": bytes_feature(tf.io.serialize_tensor(bbox)),
        "label": bytes_feature(tf.io.serialize_tensor(np.array(label))),
    })
    return tf.train.Example(features = features)

In [10]:
def create_tf_record(group_x, path, writer):
    i = 0
    for group in group_x:
        tf_ex = create_tf_ex(group, path)
        writer.write(tf_ex.SerializeToString())
        i += 1
    writer.close()
    print("DONE {}\n{}".format(path, i))

In [11]:
path_train = "../dataset/train/augmented"
path_test = "../dataset/test/augmented"

train_out = "./train.tfrecord"
test_out = "./test.tfrecord"

writer_train = tf.io.TFRecordWriter(train_out)
writer_test = tf.io.TFRecordWriter(test_out)

df_train = xml_to_csv(path_train)
df_test = xml_to_csv(path_test)

group_train = regroup(df_train)
group_test = regroup(df_test)

1.Pothole.jpg
1.Pothole_90lRotate.jpg
1.Pothole_90rRotate.jpg
1.Pothole_horizontalFlip.jpg
1.Pothole_verticalFlip.jpg
1.jpg
1.jpg
1.jpg
1.jpg
1.jpg
10.jpg
10.jpg
10._106607865_65893a7f-c27f-4be9-96cd-56f709475d40.jpg
10._106607865_65893a7f-c27f-4be9-96cd-56f709475d40_90lRotate.jpg
10._106607865_65893a7f-c27f-4be9-96cd-56f709475d40_90rRotate.jpg
10._106607865_65893a7f-c27f-4be9-96cd-56f709475d40_horizontalFlip.jpg
10._106607865_65893a7f-c27f-4be9-96cd-56f709475d40_verticalFlip.jpg
100.0_Pothole-in-road.jpg
100.0_Pothole-in-road_90lRotate.jpg
100.0_Pothole-in-road_90rRotate.jpg
100.0_Pothole-in-road_horizontalFlip.jpg
100.0_Pothole-in-road_verticalFlip.jpg
101.pothole_ddcba8e4-a344-11e9-85f3-0f8400bbe260.jpg
101.pothole_ddcba8e4-a344-11e9-85f3-0f8400bbe260_90lRotate.jpg
101.pothole_ddcba8e4-a344-11e9-85f3-0f8400bbe260_90rRotate.jpg
101.pothole_ddcba8e4-a344-11e9-85f3-0f8400bbe260_horizontalFlip.jpg
101.pothole_ddcba8e4-a344-11e9-85f3-0f8400bbe260_verticalFlip.jpg
102.5bf2a7765728d.jpg
10

12.Pothole_horizontalFlip.jpg
12.Pothole_horizontalFlip.jpg
12.Pothole_verticalFlip.jpg
12.Pothole_verticalFlip.jpg
12.Pothole_verticalFlip.jpg
12.Pothole_verticalFlip.jpg
12.jpg
12.jpg
12.jpg
12.jpg
12.jpg
12.jpg
12.jpg
12.jpg
12.jpg
120.636554993224215719-INI1Brd-02-24-2018-Star-1-A001-2018-02-23-IMG-INI1Brd-02-18-2018-S-1-1-KML931O2-L1186531823-IMG-INI1Brd-02-18-2018-S-1-1-KML931O2.jpg
120.636554993224215719-INI1Brd-02-24-2018-Star-1-A001-2018-02-23-IMG-INI1Brd-02-18-2018-S-1-1-KML931O2-L1186531823-IMG-INI1Brd-02-18-2018-S-1-1-KML931O2_90lRotate.jpg
120.636554993224215719-INI1Brd-02-24-2018-Star-1-A001-2018-02-23-IMG-INI1Brd-02-18-2018-S-1-1-KML931O2-L1186531823-IMG-INI1Brd-02-18-2018-S-1-1-KML931O2_90rRotate.jpg
120.636554993224215719-INI1Brd-02-24-2018-Star-1-A001-2018-02-23-IMG-INI1Brd-02-18-2018-S-1-1-KML931O2-L1186531823-IMG-INI1Brd-02-18-2018-S-1-1-KML931O2_horizontalFlip.jpg
120.636554993224215719-INI1Brd-02-24-2018-Star-1-A001-2018-02-23-IMG-INI1Brd-02-18-2018-S-1-1-KML931O2

134.PotholeStory1_horizontalFlip.jpg
134.PotholeStory1_verticalFlip.jpg
135.Pothole-2018-Copy.jpg
135.Pothole-2018-Copy.jpg
135.Pothole-2018-Copy_90lRotate.jpg
135.Pothole-2018-Copy_90lRotate.jpg
135.Pothole-2018-Copy_90rRotate.jpg
135.Pothole-2018-Copy_90rRotate.jpg
135.Pothole-2018-Copy_horizontalFlip.jpg
135.Pothole-2018-Copy_horizontalFlip.jpg
135.Pothole-2018-Copy_verticalFlip.jpg
135.Pothole-2018-Copy_verticalFlip.jpg
136.0a957e0b420a4f1ea782434d46ae371a.jpg
136.0a957e0b420a4f1ea782434d46ae371a.jpg
136.0a957e0b420a4f1ea782434d46ae371a_90lRotate.jpg
136.0a957e0b420a4f1ea782434d46ae371a_90lRotate.jpg
136.0a957e0b420a4f1ea782434d46ae371a_90rRotate.jpg
136.0a957e0b420a4f1ea782434d46ae371a_90rRotate.jpg
136.0a957e0b420a4f1ea782434d46ae371a_horizontalFlip.jpg
136.0a957e0b420a4f1ea782434d46ae371a_horizontalFlip.jpg
136.0a957e0b420a4f1ea782434d46ae371a_verticalFlip.jpg
136.0a957e0b420a4f1ea782434d46ae371a_verticalFlip.jpg
139.5c7dcb05cde6b.image.jpg
139.5c7dcb05cde6b.image.jpg
139.5c7dcb

147.694940094001_6021619486001_6021618077001-vs_90rRotate.jpg
147.694940094001_6021619486001_6021618077001-vs_horizontalFlip.jpg
147.694940094001_6021619486001_6021618077001-vs_verticalFlip.jpg
148.shutterstock_1043364970.jpg
148.shutterstock_1043364970_90lRotate.jpg
148.shutterstock_1043364970_90rRotate.jpg
148.shutterstock_1043364970_horizontalFlip.jpg
148.shutterstock_1043364970_verticalFlip.jpg
14_90lRotate.jpg
14_90lRotate.jpg
14_90lRotate.jpg
14_90lRotate.jpg
14_90lRotate.jpg
14_90lRotate.jpg
14_90rRotate.jpg
14_90rRotate.jpg
14_90rRotate.jpg
14_90rRotate.jpg
14_90rRotate.jpg
14_90rRotate.jpg
14_horizontalFlip.jpg
14_horizontalFlip.jpg
14_horizontalFlip.jpg
14_horizontalFlip.jpg
14_horizontalFlip.jpg
14_horizontalFlip.jpg
14_verticalFlip.jpg
14_verticalFlip.jpg
14_verticalFlip.jpg
14_verticalFlip.jpg
14_verticalFlip.jpg
14_verticalFlip.jpg
15.jpg
15.jpg
15.jpg
15.jpg
15.jpg
15.jpg
151.5DSCWFBOZZC7PNMBJCHQHAGCVI.jpg
151.5DSCWFBOZZC7PNMBJCHQHAGCVI.jpg
151.5DSCWFBOZZC7PNMBJCHQHAGCVI

177.stream_img.jpg
177.stream_img.jpg
177.stream_img_90lRotate.jpg
177.stream_img_90lRotate.jpg
177.stream_img_90rRotate.jpg
177.stream_img_90rRotate.jpg
177.stream_img_horizontalFlip.jpg
177.stream_img_horizontalFlip.jpg
177.stream_img_verticalFlip.jpg
177.stream_img_verticalFlip.jpg
179.pothole-uk.jpg
179.pothole-uk_90lRotate.jpg
179.pothole-uk_90rRotate.jpg
179.pothole-uk_horizontalFlip.jpg
179.pothole-uk_verticalFlip.jpg
18.https%3A%2F%2Fblogs-images.forbes.com%2Flaurenfix%2Ffiles%2F2018%2F04%2FPothole-damage.jpg
18.https%3A%2F%2Fblogs-images.forbes.com%2Flaurenfix%2Ffiles%2F2018%2F04%2FPothole-damage.jpg
18.https%3A%2F%2Fblogs-images.forbes.com%2Flaurenfix%2Ffiles%2F2018%2F04%2FPothole-damage.jpg
18.https%3A%2F%2Fblogs-images.forbes.com%2Flaurenfix%2Ffiles%2F2018%2F04%2FPothole-damage.jpg
18.https%3A%2F%2Fblogs-images.forbes.com%2Flaurenfix%2Ffiles%2F2018%2F04%2FPothole-damage.jpg
18.https%3A%2F%2Fblogs-images.forbes.com%2Flaurenfix%2Ffiles%2F2018%2F04%2FPothole-damage.jpg
18.http

...

test_0193_90rRotate.jpg
test_0193_horizontalFlip.jpg
test_0193_verticalFlip.jpg
test_0203.jpg
test_0203_90lRotate.jpg
test_0203_90rRotate.jpg
test_0203_horizontalFlip.jpg
test_0203_verticalFlip.jpg
test_0251.jpg
test_0251.jpg
test_0251_90lRotate.jpg
test_0251_90lRotate.jpg
test_0251_90rRotate.jpg
test_0251_90rRotate.jpg
test_0251_horizontalFlip.jpg
test_0251_horizontalFlip.jpg
test_0251_verticalFlip.jpg
test_0251_verticalFlip.jpg
test_0252.jpg
test_0252.jpg
test_0252_90lRotate.jpg
test_0252_90lRotate.jpg
test_0252_90rRotate.jpg
test_0252_90rRotate.jpg
test_0252_horizontalFlip.jpg
test_0252_horizontalFlip.jpg
test_0252_verticalFlip.jpg
test_0252_verticalFlip.jpg
test_0274.jpg
test_0274_90lRotate.jpg
test_0274_90rRotate.jpg
test_0274_horizontalFlip.jpg
test_0274_verticalFlip.jpg
test_0275.jpg
test_0275.jpg
test_0275_90lRotate.jpg
test_0275_90lRotate.jpg
test_0275_90rRotate.jpg
test_0275_90rRotate.jpg
test_0275_horizontalFlip.jpg
test_0275_horizontalFlip.jpg
test_0275_verticalFlip.jpg
tes

125.3059151_020918-wpvi-pothole-problems-4pm-video-vid.jpg
125.3059151_020918-wpvi-pothole-problems-4pm-video-vid.jpg
125.3059151_020918-wpvi-pothole-problems-4pm-video-vid_90lRotate.jpg
125.3059151_020918-wpvi-pothole-problems-4pm-video-vid_90lRotate.jpg
125.3059151_020918-wpvi-pothole-problems-4pm-video-vid_90rRotate.jpg
125.3059151_020918-wpvi-pothole-problems-4pm-video-vid_90rRotate.jpg
125.3059151_020918-wpvi-pothole-problems-4pm-video-vid_horizontalFlip.jpg
125.3059151_020918-wpvi-pothole-problems-4pm-video-vid_horizontalFlip.jpg
125.3059151_020918-wpvi-pothole-problems-4pm-video-vid_verticalFlip.jpg
125.3059151_020918-wpvi-pothole-problems-4pm-video-vid_verticalFlip.jpg
127.shutterstock_1060794287-1523468315-7078.jpg
127.shutterstock_1060794287-1523468315-7078_90lRotate.jpg
127.shutterstock_1060794287-1523468315-7078_90rRotate.jpg
127.shutterstock_1060794287-1523468315-7078_horizontalFlip.jpg
127.shutterstock_1060794287-1523468315-7078_verticalFlip.jpg
13.potholes-magnum.jpg
13.

176.Pothole_verticalFlip.jpg
176.Pothole_verticalFlip.jpg
178.potholes-the-true-depth-article-image.jpg
178.potholes-the-true-depth-article-image.jpg
178.potholes-the-true-depth-article-image.jpg
178.potholes-the-true-depth-article-image.jpg
178.potholes-the-true-depth-article-image_90lRotate.jpg
178.potholes-the-true-depth-article-image_90lRotate.jpg
178.potholes-the-true-depth-article-image_90lRotate.jpg
178.potholes-the-true-depth-article-image_90lRotate.jpg
178.potholes-the-true-depth-article-image_90rRotate.jpg
178.potholes-the-true-depth-article-image_90rRotate.jpg
178.potholes-the-true-depth-article-image_90rRotate.jpg
178.potholes-the-true-depth-article-image_90rRotate.jpg
178.potholes-the-true-depth-article-image_horizontalFlip.jpg
178.potholes-the-true-depth-article-image_horizontalFlip.jpg
178.potholes-the-true-depth-article-image_horizontalFlip.jpg
178.potholes-the-true-depth-article-image_horizontalFlip.jpg
178.potholes-the-true-depth-article-image_verticalFlip.jpg
178.pot

44.0ea0e8797ce553e1f2fef81bff857868_XL_90lRotate.jpg
44.0ea0e8797ce553e1f2fef81bff857868_XL_90lRotate.jpg
44.0ea0e8797ce553e1f2fef81bff857868_XL_90lRotate.jpg
44.0ea0e8797ce553e1f2fef81bff857868_XL_90lRotate.jpg
44.0ea0e8797ce553e1f2fef81bff857868_XL_90rRotate.jpg
44.0ea0e8797ce553e1f2fef81bff857868_XL_90rRotate.jpg
44.0ea0e8797ce553e1f2fef81bff857868_XL_90rRotate.jpg
44.0ea0e8797ce553e1f2fef81bff857868_XL_90rRotate.jpg
44.0ea0e8797ce553e1f2fef81bff857868_XL_horizontalFlip.jpg
44.0ea0e8797ce553e1f2fef81bff857868_XL_horizontalFlip.jpg
44.0ea0e8797ce553e1f2fef81bff857868_XL_horizontalFlip.jpg
44.0ea0e8797ce553e1f2fef81bff857868_XL_horizontalFlip.jpg
44.0ea0e8797ce553e1f2fef81bff857868_XL_verticalFlip.jpg
44.0ea0e8797ce553e1f2fef81bff857868_XL_verticalFlip.jpg
44.0ea0e8797ce553e1f2fef81bff857868_XL_verticalFlip.jpg
44.0ea0e8797ce553e1f2fef81bff857868_XL_verticalFlip.jpg
59.Potholes.png
59.Potholes_90lRotate.png
59.Potholes_90rRotate.png
59.Potholes_horizontalFlip.png
59.Potholes_verticalF

99.penis-potholes.png
99.penis-potholes_90lRotate.png
99.penis-potholes_90rRotate.png
99.penis-potholes_horizontalFlip.png
99.penis-potholes_verticalFlip.png


In [12]:
create_tf_record(group_train, path_train, writer_train)
create_tf_record(group_test, path_test, writer_test)

DONE ../dataset/train/augmented
1780
DONE ../dataset/test/augmented
295
